## Assignment 2: Nearest Neighbours & Decision Trees

In [2]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import sklearn


In [3]:
raw_df = pd.read_csv("forestCover.csv")
display(raw_df)

,Elevation,Aspect,Facet,Slope,Inclination,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,...,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Water_Level,Observation_ID,Cover_Type
0,3208920,166,415.394727,6,0.691628,408,62,3137,225,242,...,0,0,0,0,0,0,0,2,1,2
1,2789020,137,343.302186,9,0.621245,30,2,1040,235,237,...,0,0,0,0,0,0,0,3,1,3
2,3384615,357,894.231390,9,-0.266086,362,15,3113,207,225,...,0,0,0,0,0,0,0,4,1,1
3,3348150,148,371.346939,6,0.786375,247,50,1207,228,240,...,0,0,0,0,0,0,0,5,1,2
4,3061955,124,310.783430,11,-0.335685,170,19,1595,238,232,...,0,0,0,0,0,0,0,6,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,3236545,117,293.274632,17,0.257413,60,0,626,247,222,...,0,0,0,0,0,0,0,581009,1,2
581008,3751475,321,805.501719,6,-0.479762,272,19,2906,204,234,...,0,0,0,0,0,1,0,581010,1,1
581009,2855320,72,180.453289,6,0.946018,0,0,1719,227,228,...,0,0,0,0,0,0,0,581011,1,6
581010,3004495,100,250.901923,7,0.113070,0,0,2067,231,231,...,0,0,0,0,0,0,0,581012,1,2
